# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-02-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056
[['Barry Tuckwell/Academy of St Martin-in-the-Fields/Sir Neville Marriner', 'Logged In', 'Mohammad', 'M', '0', 'Rodriguez', '277.15873', 'paid', 'Sacramento--Roseville--Arden-Arcade, CA', 'PUT', 'NextSong', '1.54051E+12', '961', 'Horn Concerto No. 4 in E flat K495: II. Romance (Andante cantabile)', '200', '1.54328E+12', '88'], ['Jimi Hendrix', 'Logged In', 'Mohammad', 'M', '1', 'Rodriguez', '239.82975', 'paid', 'Sacramento--Roseville--Arden-Arcade, CA', 'PUT', 'NextSong', '1.54051E+12', '961', 'Woodstock Inprovisation', '200', '1.54328E+12', '88'], ['Building 429', 'Logged In', 'Mohammad', 'M', '2', 'Rodriguez', '300.61669', 'paid', 'Sacramento--Roseville--Arden-Arcade, CA', 'PUT', 'NextSong', '1.54051E+12', '961', 'Majesty (LP Version)', '200', '1.54328E+12', '88'], ["The B-52's", 'Logged In', 'Gianna', 'F', '0', 'Jones', '321.54077', 'free', 'New York-Newark-Jersey City, NY-NJ-PA', 'PUT', 'NextSong', '1.54087E+12', '107', 'Love Shack', '200', '1.54328E+12', '38'], ['Die Mooskirc

In [2]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))
#SAVE EVENT CSV AS DATAFRAME 
df=pd.read_csv('event_datafile_new.csv')
#SHOW DATAFRAME
df.head()

6821


NameError: name 'pd' is not defined

# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# CREATE CLUSTER NODE 

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

CREATE FIRST TABLE AND INSERT DATA

In [8]:
#session_id and item_in_session columns are partition keys
#create table and execute creation
query = "CREATE TABLE IF NOT EXISTS session_song "
query=query+"(      session_id INT, \
                    item_in_session INT, \
                    artist text, \
                    song text, \
                    length float, \ 
                    PRIMARY KEY (session_id,item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [20]:
#insert
for _,row in df.iterrows():
    query="INSERT into session_song (session_id, item_in_session, artist, song,length)"
    query=query+"VALUES (%s,%s,%s,%s,%s)"
    session.execute(query,(row["sessionId"],row["itemInSession"],row["artist"],row["song"],row["length"]))

Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4.

In [14]:
#write query and execute
query="SELECT artist,song,length FROM session_song WHERE session_id = 338 AND item_in_session = 4"
try:
    rows=session.execute(query)
except Exception as e:
    print(e)
#iterate through created resultset
for row in rows:
    print(row.artist,row.song,row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


CREATE SECOND TABLE AND INSERT DATA

In [15]:
#session_id and user_id are partition keys item_in_session is clustering key
#create table and execute creation
query = "CREATE TABLE IF NOT EXISTS artist_song_user "
query = query + "(item_in_session int, \
                  session_id int, \
                  artist text, \
                  song text, \
                  first_name text, \
                  last_name text, \
                  user_id int, \
                  PRIMARY KEY ((session_id,  user_id),item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)
#insert
for _,row in df.iterrows():
    query="INSERT INTO artist_song_user(item_in_session,session_id,artist,song,first_name,last_name,user_id)"
    query=query+"VALUES (%s,%s,%s,%s,%s,%s,%s)"
    session.execute(query,(row["itemInSession"],row["sessionId"],row["artist"],row["song"],row["firstName"],row["lastName"],row["userId"]))

    

PERFORM SECOND QUERY
Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182.

In [16]:
#write query and execute
query="SELECT artist,song,first_name,last_name FROM artist_song_user WHERE session_id = 182 AND user_id = 10"
try:
    rows=session.execute(query)
except Exception as e:
    print(e)
#iterate through created resultset 
for row in rows:
    print(row.artist,row.song,row.first_name,row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


CREATE THIRD TABLE user_name AND INSERT DATA

In [17]:
#the song column is partition key and user_id column is clustering column
#create table and execute creation
query="CREATE TABLE IF NOT EXISTS user_name(song text, \
                                            first_name text, \
                                            last_name text, \
                                            user_id int, \
                                            PRIMARY KEY (song,user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)
#insert
for _,row in df.iterrows():
    query="INSERT INTO user_name(song,first_name,last_name,user_id) VALUES (%s,%s,%s,%s)"
    session.execute(query,(row["song"],row["firstName"],row["lastName"],row["userId"]))

PERFORM THIRD QUERY
Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'.

In [18]:
#write query and execute
query="select first_name,last_name from user_name WHERE song='All Hands Against His Own'"
try:
    rows=session.execute(query)
except Exception as e:
    print(e)
#iterate through created resultset 
for row in rows:
    print(row.first_name,row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [21]:
#DROP TABLES
session.execute("DROP TABLE session_song")
session.execute("DROP TABLE artist_song_user")
session.execute("DROP TABLE user_name")

### Close the session and cluster connection¶

In [21]:
#CLOSE CONNECTION
session.shutdown()
cluster.shutdown()